# Clumping and Thresholding (C + T)
## Data
- Superpopulations: 'EUR' and 'AFR'
- Populations (similar to PS4)
  - EUR: CEU (training), GBR (validation), FIN (testing)
  - AFR: YRI (training), ESN (validation), GWD (testing)
- Genotypes: Chromosome 19 (1000 Genomes)
- Phenotypes (LDAK Simulation)
  - Power: -0.25, -1
  - Heritability: 0.1, 0.3, 0.5, 0.7, 0.9
  - Number of Causal SNPs: 1, 10, 100, 250, 512
  - Number of Phenotypes (Traits): 20
  - Effect Sizes: fixed (use LDL PRS data)

## References
- Clumping and Thresholding: UCSD CSE 284 Lecture Slides, PS3, and PS4
- Phenotype Simulation: https://dougspeed.com/simulations
- LDL PRS Data: https://www.nature.com/articles/s41586-021-04064-3


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
! wget https://s3.amazonaws.com/plink2-assets/alpha5/plink2_linux_x86_64_20240105.zip
! unzip plink2_linux_x86_64_20240105.zip
! ./plink2

--2024-03-11 18:27:57--  https://s3.amazonaws.com/plink2-assets/alpha5/plink2_linux_x86_64_20240105.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.34.30, 16.182.109.40, 52.216.54.16, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.34.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9266484 (8.8M) [application/zip]
Saving to: ‘plink2_linux_x86_64_20240105.zip’

plink2_linux_x86_64 100%[===================>]   8.84M  31.7MB/s    in 0.3s    

2024-03-11 18:27:58 (31.7 MB/s) - ‘plink2_linux_x86_64_20240105.zip’ saved [9266484/9266484]

Archive:  plink2_linux_x86_64_20240105.zip
  inflating: plink2                  
PLINK v2.00a5.10LM 64-bit Intel (5 Jan 2024)   www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3

  plink2 <input flag(s)...> [command flag(s)...] [other flag(s)...]
  plink2 --help [flag name(s)...]

Commands include --rm-dup list, --make-bpgen, --export, --freq, -

In [11]:
# Import libraries
%pylab inline
import os
import csv
import scipy.stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [17]:
! cd /content/drive/MyDrive/CSE-284-Final-Project/data/CT/superpopulation/ | ls -1 | wc -l

635


In [12]:
# POP_LIST = ['EUR', 'AFR']
# POWER_LIST = [-0.25, -1]
# HER_LIST = [0.1, 0.3, 0.5, 0.7, 0.9]
# NUM_CAUSALS_LIST = [1, 10, 100, 250, 512]
# PHENO_NUM_START = 1
# PHENO_NUM_END = 20


POP_LIST = ['EUR']
POP_TRAIN = 'CEU'
POP_VAL = 'GBR'
POP_TEST = 'FIN'
POWER_LIST = [-0.25, -1]
HER_LIST = [0.1, 0.3, 0.5, 0.7, 0.9]
NUM_CAUSALS_LIST = [1, 10, 100, 250, 512]
PHENO_NUM_START = 1
PHENO_NUM_END = 20

In [14]:
! cp /content/drive/MyDrive/CSE-284-Final-Project/data/1000g_by_chrom/chr19.* .
# ! cp /content/drive/MyDrive/CSE-284-Final-Project/data/1000g_by_population/{POP_TRAIN}_all* .
# ! cp /content/drive/MyDrive/CSE-284-Final-Project/data/1000g_by_population/{POP_VAL}_all* .
# ! cp /content/drive/MyDrive/CSE-284-Final-Project/data/1000g_by_population/{POP_TEST}_all* .

PREFIX = '/content'
chr19 = f'{PREFIX}/chr19'

! echo "0.000001 0 0.000001" > range_list
! echo "0.00001 0 0.00001" >> range_list
! echo "0.0001 0 0.0001" >> range_list
! echo "0.001 0 0.001" >> range_list
! echo "0.05 0 0.05" >> range_list
! echo "0.1 0 0.1" >> range_list

for POP in POP_LIST:

  ! cp /content/drive/MyDrive/CSE-284-Final-Project/data/split_ids/by_population/{POP_TRAIN}_all.txt .
  ! cp /content/drive/MyDrive/CSE-284-Final-Project/data/split_ids/by_population/{POP_VAL}_all.txt .
  ! cp /content/drive/MyDrive/CSE-284-Final-Project/data/split_ids/by_population/{POP_TEST}_all.txt .
  ! cut -d ' ' -f 2 {POP_TRAIN}_all.txt > {POP}_train_IID_only.txt
  ! cut -d ' ' -f 2 {POP_VAL}_all.txt > {POP}_val_IID_only.txt
  ! cut -d ' ' -f 2 {POP_TEST}_all.txt > {POP}_test_IID_only.txt

  keep_train = f'{POP}_train_IID_only.txt'
  out_train = f'{POP}_train'

  keep_val = f'{POP}_val_IID_only.txt'
  out_val = f'{POP}_val'

  keep_test = f'{POP}_test_IID_only.txt'
  out_test = f'{POP}_test'

  # Make bed

  sh = f'''
  ./plink2 \
    --make-bed \
    --bfile {chr19} \
    --keep {keep_train} \
    --out {out_train}

  ./plink2 \
    --make-bed \
    --bfile {chr19} \
    --keep {keep_val} \
    --out {out_val}

  ./plink2 \
    --make-bed \
    --bfile {chr19} \
    --keep {keep_test} \
    --out {out_test}

  '''
  with open('makebed.sh', 'w') as file:
      file.write(sh)
  !bash makebed.sh

  GENO_TRAIN = f'{PREFIX}/{POP}_train'
  GENO_VAL = f'{PREFIX}/{POP}_val'
  GENO_TEST = f'{PREFIX}/{POP}_test'

  for POWER in POWER_LIST:
    for HER in HER_LIST:
      for NUM_CAUSALS in NUM_CAUSALS_LIST:
        ! cp /content/drive/MyDrive/CSE-284-Final-Project/data/chr19_ldl_pheno/power={POWER}_her={HER}_num-causals={NUM_CAUSALS}.pheno .
        PHENO = f'{PREFIX}/power={POWER}_her={HER}_num-causals={NUM_CAUSALS}.pheno'

        # GWAS (GLM)
        sh = f'''
        ./plink2 --bfile {GENO_TRAIN} \
                --pheno {PHENO} \
                --glm allow-no-covars \
                --maf 0.05 \
                --out {POP}_gwas \
                --silent
        '''

        with open('gwas.sh', 'w') as file:
            file.write(sh)
        !bash gwas.sh


        combined_prediction_data = []
        combined_result_data = []

        for PHENO_NUM in range(PHENO_NUM_START, PHENO_NUM_END + 1):

            GLM = f'/content/{POP}_gwas.PHENO{PHENO_NUM}.glm.linear'

            # Clumping

            sh = f'''
            ./plink2 \
                --bfile {GENO_TRAIN} \
                --clump-p1 1 \
                --clump-r2 0.1 \
                --clump-kb 250 \
                --clump {GLM} \
                --out {POP}_gwas.PHENO{PHENO_NUM} \
                --silent

            awk 'NR!=1{{print $3}}' {POP}_gwas.PHENO{PHENO_NUM}.clumps > {POP}_gwas.PHENO{PHENO_NUM}.valid.snp
            cat {POP}_gwas.PHENO{PHENO_NUM}.clumps | awk '{{print $3 "\t" $4}}' > {POP}_gwas.PHENO{PHENO_NUM}.pvals
            '''

            with open('clump.sh', 'w') as file:
                file.write(sh)
            !bash clump.sh

            GLM = f'/content/{POP}_gwas.PHENO{PHENO_NUM}.glm.linear'
            PVALS = f'/content/{POP}_gwas.PHENO{PHENO_NUM}.pvals'
            VAL = f'/content/{POP}_gwas.PHENO{PHENO_NUM}.valid.snp'

            # Scoring

            sh = f'''

            ./plink2 \
            --bfile {GENO_TRAIN} \
            --score {GLM} 3 7 12 header cols=+scoresums\
            --q-score-range range_list {PVALS} \
            --extract {VAL} \
            --out {POP}_PHENO{PHENO_NUM}_train \
            --silent

            ./plink2 \
            --bfile {GENO_VAL} \
            --score {GLM} 3 7 12 header cols=+scoresums\
            --q-score-range range_list {PVALS} \
            --extract {VAL} \
            --out {POP}_PHENO{PHENO_NUM}_val \
            --silent

            ./plink2 \
            --bfile {GENO_TEST} \
            --score {GLM} 3 7 12 header cols=+scoresums\
            --q-score-range range_list {PVALS} \
            --extract {VAL} \
            --out {POP}_PHENO{PHENO_NUM}_test \
            --silent
            '''

            with open('score.sh', 'w') as file:
              file.write(sh)
            !bash score.sh

            # Thresholding

            pval_list = ["0.000001", "0.00001", "0.0001", "0.001", "0.05", "0.1"]
            best_pval_idx = 0
            train_scores = []
            val_scores = []
            test_scores = []
            train_score = 0
            val_score = 0
            test_score = 0

            print("Training: ")
            for pval in pval_list:
                score_file = f'{POP}_PHENO{PHENO_NUM}_train.{pval}.sscore'
                if os.path.isfile(score_file) == False:
                  continue
                prs = pd.read_csv(score_file, delim_whitespace=True)
                phen = pd.read_csv(f'{PHENO}',
                              delim_whitespace=True, usecols=[0, 1, PHENO_NUM + 1], names=["FID","IID","phen"])
                d = pd.merge(prs, phen, on=["IID"])
                score = scipy.stats.pearsonr(d["phen"], d["SCORE1_SUM"])[0] ** 2
                train_scores.append(score)
                # print("pval=%s, R2=%s"%(pval, score))
            train_score = np.nanmax(train_scores)
            print(train_score)

            print("Validation: ")
            for pval in pval_list:
                score_file = f'{POP}_PHENO{PHENO_NUM}_val.{pval}.sscore'
                if os.path.isfile(score_file) == False:
                  val_scores.append(-1)
                  continue
                prs = pd.read_csv(score_file, delim_whitespace=True)
                phen = pd.read_csv(f'{PHENO}',
                              delim_whitespace=True, usecols=[0, 1, PHENO_NUM + 1], names=["FID","IID","phen"])
                d = pd.merge(prs, phen, on=["IID"])
                score = scipy.stats.pearsonr(d["phen"], d["SCORE1_SUM"])[0] ** 2
                val_scores.append(score)
                # print("pval=%s, R2=%s"%(pval, score))
            val_score = np.nanmax(val_scores)
            best_pval_idx = val_scores.index(val_score)
            print("Best: ", pval_list[best_pval_idx], val_score)

            print("Testing: ")
            best_pval = pval_list[best_pval_idx]
            prs = pd.read_csv(f'{POP}_PHENO{PHENO_NUM}_test.{best_pval}.sscore', delim_whitespace=True)
            phen = pd.read_csv(f'{PHENO}',
                              delim_whitespace=True, usecols=[0, 1, PHENO_NUM + 1], names=["FID","IID","phen"])
            d = pd.merge(prs, phen, on=["IID"])
            test_score = scipy.stats.pearsonr(d["phen"], d["SCORE1_SUM"])[0] ** 2
            print("pval=%s, R2=%s"%(best_pval, test_score))
            predict_df = d[['IID', 'SCORE1_SUM', 'phen']].copy()
            predict_df.rename(columns={'IID': 'IID', 'SCORE1_SUM': 'predict', 'phen': 'actual'}, inplace=True)
            predict_df["pheno_num"] = PHENO_NUM
            combined_prediction_data.append(predict_df)

            result_dict = {
                "population/superpopulation": POP,
                "power": POWER,
                "her": HER,
                "num_causals": NUM_CAUSALS,
                "pheno_num": PHENO_NUM,
                "train": train_score,
                "val": val_score,
                "test": test_score
            }
            combined_result_data.append(result_dict)

        combined_prediction_df = pd.concat(combined_prediction_data, ignore_index=True)
        combined_result_df = pd.DataFrame(combined_result_data)

        # Write combined prediction data to file
        combined_prediction_file = f'combined_predict_{POP}_power={POWER}_her={HER}_num-causals={NUM_CAUSALS}_pheno={PHENO_NUM_START}-{PHENO_NUM_END}.csv'
        combined_prediction_df.to_csv(combined_prediction_file, index=False)
        print("Combined prediction data has been written to:", combined_prediction_file)

        # Write combined result data to file
        combined_result_file = f'combined_result_{POP}_power={POWER}_her={HER}_num-causals={NUM_CAUSALS}_pheno={PHENO_NUM_START}-{PHENO_NUM_END}.csv'
        combined_result_df.to_csv(combined_result_file, index=False)
        print("Combined result data has been written to:", combined_result_file)

        print(combined_prediction_df)
        print(combined_result_df)

        ! cp combined_predict_{POP}_power={POWER}_her={HER}_num-causals={NUM_CAUSALS}_pheno={PHENO_NUM_START}-{PHENO_NUM_END}.csv /content/drive/MyDrive/CSE-284-Final-Project/data/CT/population
        ! cp combined_result_{POP}_power={POWER}_her={HER}_num-causals={NUM_CAUSALS}_pheno={PHENO_NUM_START}-{PHENO_NUM_END}.csv /content/drive/MyDrive/CSE-284-Final-Project/data/CT/population



PLINK v2.00a5.10LM 64-bit Intel (5 Jan 2024)   www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to EUR_train.log.
Options in effect:
  --bfile /content/chr19
  --keep EUR_train_IID_only.txt
  --make-bed
  --out EUR_train

Start time: Mon Mar 11 18:29:26 2024
12978 MiB RAM detected, ~11767 available; reserving 6489 MiB for main
workspace.
Using up to 2 compute threads.
2504 samples (1270 females, 1234 males; 2497 founders) loaded from
/content/chr19.fam.
283173 variants loaded from /content/chr19.bim.
Note: No phenotype data present.
--keep: 99 samples remaining.
99 samples (50 females, 49 males; 99 founders) remaining after main filters.
Writing EUR_train.fam ... done.
Writing EUR_train.bim ... done.
Writing EUR_train.bed ... 0%23%46%69%92%done.
End time: Mon Mar 11 18:29:27 2024
PLINK v2.00a5.10LM 64-bit Intel (5 Jan 2024)   www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang